# Assess & Clean Data

Load raw data, assess, clean and save in proper sets.


### Data Sources

- portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
- profile.json - demographic data for each customer
- transcript.json - records for transactions, offers received, offers viewed, and offers completed

### Changes

- 2018-12-19: Started project



In [3]:
# load libraries

import numpy as np
import pandas as pd
from tqdm import tqdm

# my own custom functions
import EDA_functions as EDA
import cleaning_functions as cleaning

# visualization
import matplotlib.pyplot as plt
import seaborn as sns #, sns.set_style('whitegrid')
color = 'rebeccapurple'
%matplotlib inline

# display settings
from IPython.display import display
pd.options.display.max_columns = None

from pathlib import Path  # to make file path references relative to notebook directory

In [4]:
# import data

portfolio_file = Path.cwd() / "data" / "raw" / "portfolio.json"
profile_file = Path.cwd() / "data" / "raw" / "profile.json"
transcript_file = Path.cwd() / "data" / "raw" / "transcript.json"

portfolio = pd.read_json(portfolio_file, orient='records', lines=True)
profile = pd.read_json(profile_file, orient='records', lines=True)
transcript = pd.read_json(transcript_file, orient='records', lines=True)

### Check portfolio data

In [6]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
channels      10 non-null object
difficulty    10 non-null int64
duration      10 non-null int64
id            10 non-null object
offer_type    10 non-null object
reward        10 non-null int64
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes


In [7]:
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


Explanations: 
- id (string) - offer id
- offer_type (string) - type of offer ie BOGO (buy-one-get-one-free), discount, informational
- difficulty (int) - minimum required spend to complete an offer
- reward (int) - reward given for completing an offer
- duration (int) - 
- channels (list of strings)

### Check profile data

In [9]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
age                 17000 non-null int64
became_member_on    17000 non-null int64
gender              14825 non-null object
id                  17000 non-null object
income              14825 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.1+ KB


In [10]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


Explanations:

* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

### Check transcript data

In [12]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
event     306534 non-null object
person    306534 non-null object
time      306534 non-null int64
value     306534 non-null object
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [22]:
transcript.sample(100)

,event,person,time,value
94151,offer completed,8296312262cd4cc19872e53ad1e10345,246,{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2...
49107,transaction,81e6836cba764009a168d84bb936c56a,138,{'amount': 1.3599999999999999}
258251,offer completed,8e508511652c446a9debf0c20f5f5163,576,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...
177163,transaction,f3934f05d51f47c7a470661cbb774075,426,{'amount': 8.57}
19934,transaction,e5d3238906284a18814d35787c9df9d8,12,{'amount': 0.39}
93921,transaction,cf0ee2e478b7446c9c07a63c35508464,246,{'amount': 13.83}
29867,transaction,e907946cc01e4e11b814b1ab946755a0,48,{'amount': 2.14}
29197,transaction,edcddc3f57f34f09a049559d2606cf6b,42,{'amount': 0.44}
203885,offer received,a79f46d98c1944db936847503b1eaa29,504,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
240850,transaction,d27b2ed395fc4f1d814de65eb0815c95,558,{'amount': 21.25}


Explanations:
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours after start of 30 days test period. The data begins at time t=0
* value (dict of strings) - either an offer id or transaction amount depending on the record

**Tasklist**

Cleaning tasks:
- portfolio: rename `id` col to `offer_id`
- portfolio: one-hot-encode `channels`
- portfolio: add `prop_reward` (`income` / `difficulty`)
- portfolio add `rel_difficulty` (`difficulty` / `duration` in days)
- portfolio: add `duration_hours` col 
- profile: rename `id` col to `person_id`
- profile: change dtype of `became_member_on`to datetime
- profile: transform `became_member_on` into a duration integer (days, starting from maxdate)
- transcript: rename `person` to `person_id`
- all files: simplify ids

Further preparation tasks:
- transcript: flag every transaction if within valid period of a viewed promotion
- profile: add infos about consumption (number of purchases, amount per day etc.)
    - (could later be done per promo)